In [ ]:
%matplotlib inline
import nsidc
import datetime
import xarray as xr
import glob
import pyproj

First we download a slice in time for given frequency, resolution, polarization and ascending/descending
!!! Make sure to fill in your own user pass from https://urs.earthdata.nasa.gov/ !!!


In [ ]:
freq = '36' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
res = '25'  # can also be 12.5 or 25, although this depends on the chosen frequency as well
# freq = '36' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
HV = 'V' # Horizontal or Vertical polarisation
AD = 'A' # Ascending or descending, descending contains imagery during the day, probably showing more contrast
bounds = [(-5., 0.),
          (10., 15.),
         ]

# start_date = datetime.datetime(2002, 6, 1)
start_date = datetime.datetime(2010, 12, 15) # date after data recovery
end_date = datetime.datetime(2011, 1, 15) # date after data recovery
# url = nsidc.make_measures_url(start_date, res, freq, HV, AD)
# nsidc.make_measures_download(url, 'hesselwinsemius', 'something')
username = 'xxxx'
password = 'xxxx'
# url, success = nsidc.download_measures(freq, res, HV, AD, start_date, username, password)
fn_out_prefix = 'NSIDC_Test'
nsidc.download_measures_ts(freq, res, HV, AD, start_date, end_date, bounds, fn_out_prefix, username, password)

Check if some netcdf files were produced and written to folder (you should see it in the outputs above as well). Let's open the dataset and check the content of netcdf files and plot the first slicer of brightness temperatures.

In [ ]:
fns = glob.glob('*.nc')
fns.sort()
ds = xr.open_mfdataset(fns, combine='nested', concat_dim='time')
# ds = [xr.open_dataset(fn) for fn in fns] #, concat_dim='time') # , combine_nested=True, concat_dim='time')
print(ds)
# plot one slice as example
ds.TB[0].plot()

Let's compute some CM ratio for one POI (the first, go ahead and extend if you want).

In [ ]:
import glob
import xarray as xr
import pyproj
import nsidc

fns = glob.glob('*.nc')
fns.sort()
ds = xr.open_mfdataset(fns, combine='nested', concat_dim='time')

points_interest = [(2.4406387000000223, 6.893251099999999),
                   (2.2999999999999545, 7.099999999999999),
                   (2.4851462999999967, 8.0335995),
                   (2.0499999999999545, 7.566667)
]
proj4str = '+proj=cea +lat_0=0 +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
proj_out = pyproj.Proj('epsg:4326')
# we define a projection object for the projection used in the downloaded grids.
proj_in = pyproj.Proj(proj4str)

# here we convert the coordinates in lat-lon into the coordinate system of the downloaded grids.
points_xy = nsidc.proj_coords(points_interest, proj_out, proj_in)
points_xy

# compute CM ratio for first point only
x, y = points_xy[1]
C, M, ratio = nsidc.c_m_ratio(ds['TB'], x, y)
ratio.plot(marker='.', linewidth=0)
